<a href="https://colab.research.google.com/github/pritiyadav888/Machine-Learning-Projects/blob/main/google_vantilator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Importing DataScience libraries
import pandas as pd
import gc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm,skew
from scipy import stats
%matplotlib inline
from sklearn.linear_model import LinearRegression,Ridge,RidgeCV, ElasticNetCV, LassoCV,BayesianRidge
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler,StandardScaler,MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Predictive_Data/vantilator_google/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Predictive_Data/vantilator_google/test.csv')

In [ ]:
train.describe()

,id,breath_id,R,C,time_step,u_in,u_out,pressure
count,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06
mean,3.018000e+06,6.283886e+04,2.703618e+01,2.608072e+01,1.307225e+00,7.321615e+00,6.204493e-01,1.122041e+01
std,1.742443e+06,3.633526e+04,1.959549e+01,1.715231e+01,7.659778e-01,1.343470e+01,4.852752e-01,8.109703e+00
min,1.000000e+00,1.000000e+00,5.000000e+00,1.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,-1.895744e+00
25%,1.509001e+06,3.137700e+04,5.000000e+00,1.000000e+01,6.428995e-01,3.936623e-01,0.000000e+00,6.329607e+00
50%,3.018000e+06,6.276550e+04,2.000000e+01,2.000000e+01,1.308123e+00,4.386146e+00,1.000000e+00,7.032628e+00
75%,4.527000e+06,9.430100e+04,5.000000e+01,5.000000e+01,1.965502e+00,4.983895e+00,1.000000e+00,1.364103e+01
max,6.036000e+06,1.257490e+05,5.000000e+01,5.000000e+01,2.937238e+00,1.000000e+02,1.000000e+00,6.482099e+01


In [ ]:
train.isnull().sum().max()

0

In [ ]:
train = train.fillna(0)
test = test.fillna(0)

In [ ]:
# train_df = train.head(10000)
# train_df.info()

In [ ]:
# test_df = test.head(10000)
# test_df.info()

In [ ]:
# X array - drop target
X= train.drop(['pressure'],axis=1).values

In [ ]:
# y array - target only
y=train['pressure'].values

In [ ]:
# Split Train into Train and Validate Sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20,random_state=42)

In [ ]:
# Size of our newly split datasets 
print('Train size: %i' % X_train.shape[0])
print('Validation size: %i' % X_val.shape[0])

Train size: 4828800
Validation size: 1207200


In [ ]:
# No. of folds for Cross Validation
n_folds = 3
# Defining a function which returns the mean squared error between the cross validation score of X and y arrays
def rmse_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42)
    rmse= np.sqrt(-cross_val_score(model,X_train, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [ ]:
# Intialize the Cross Validation function with Shuffle
n_folds = 3
custom_cv = KFold(n_folds, shuffle=True, random_state=42)

In [ ]:
#define  our  GB model
GBR = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [ ]:
#fitting he model on the train set
GBR.fit(X_train,y_train)

In [ ]:
# #Cross Validation Error
# rmse_cv(GBR).mean()

In [ ]:
# R squared, RMSE and Mean Absolute Error obtained by using actual and predicted values from Validate set
print('Validation R^2: %.5f'  % r2_score(y_val,GBR.predict(X_val)))
print('Validation RMSE: %.5f\n'   % mean_squared_error(y_val,GBR.predict(X_val)))
print('Validation Mean Absolute Error: %.5f\n'   % mean_absolute_error(y_val,GBR.predict(X_val)))

In [ ]:
gc.collect()

In [ ]:
# ridgecv model
ridgecv = RidgeCV(alphas=[0.1, 1.0, 10.0],cv =3)

In [ ]:
#fitting our model on the training data
ridgecv.fit(X_train,y_train)

In [ ]:
# let's call our function to calculate the mean cross validation error
rmse_cv(ridgecv).mean()

In [ ]:
# R- squared, RMSE and Mean Absolute Error for the validation sets
print('Validation R^2: %.5f'  % r2_score(y_val,ridgecv.predict(X_val)))
print('Validation RMSE: %.5f\n'   % mean_squared_error(y_val,ridgecv.predict(X_val)))
print('Validation Mean Absolute Error: %.5f\n'   % mean_absolute_error(y_val,ridgecv.predict(X_val)))

In [ ]:
gc.collect()

In [ ]:
# kernel model
KRR = KernelRidge(alpha=3)

In [ ]:
# fitting on train
KRR.fit(X_train,y_train)

In [ ]:
#  model estimatores
print('Validation R^2: %.5f'  % r2_score(y_val,KRR.predict(X_val)))
print('Validation RMSE: %.5f\n'   % mean_squared_error(y_val,KRR.predict(X_val)))
print('Validation Mean Absolute Error: %.5f\n'   % mean_absolute_error(y_val,KRR.predict(X_val)))

In [ ]:
gc.collect()

In [ ]:
# define the model with alpha values in a numpy array with cross-validation function included
lasso = LassoCV(alphas = [1, 0.1, 0.001, 0.0005],cv =custom_cv)

In [ ]:
# fit the model
lasso.fit(X_train,y_train)

In [ ]:
# Model scoring
print('Validation R^2: %.5f'  % r2_score(y_val,lasso.predict(X_val)))
print('Validation RMSE: %.5f\n'   % mean_squared_error(y_val,lasso.predict(X_val)))
print('Validation Mean Absolute Error: %.5f\n'   % mean_absolute_error(y_val,lasso.predict(X_val)))

In [ ]:
#CV error
rmse_cv(lasso).mean()

In [ ]:
gc.collect()


In [ ]:
# defining the model with alphas and Elasticnet mixing parameter l1_ratio
Elastic = ElasticNetCV(alphas=[0.0001, 0.0005, 0.001, 0.01, 0.1, 1, 10], l1_ratio=[.01, .1, .5, .9, .99], max_iter=5000
                       ,cv=custom_cv)

In [ ]:
# fit the model
Elastic.fit(X_train,y_train)

In [ ]:
# Model scores
print('Validation R^2: %.5f'  % r2_score(y_val,Elastic.predict(X_val)))
print('Validation RMSE: %.5f\n'   % mean_squared_error(y_val,Elastic.predict(X_val)))
print('Validation Mean Absolute Error: %.5f\n'   % mean_absolute_error(y_val,Elastic.predict(X_val)))

In [ ]:
# CV error
rmse_cv(Elastic).mean()

In [ ]:
gc.collect()


In [ ]:
# import pipeline
from sklearn.pipeline import Pipeline
# initialize pipeline with the model and here we have function which takes cares of outlier
GBRPipe = Pipeline([
        ('outlier', RobustScaler()),
        ('gbm', GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5))])

In [ ]:
# set up the parameter grid with parameters you like to iterate on. Here we have 3
param_grid = {'gbm__learning_rate': [0.01, 0.05],
             'gbm__max_depth': [1,5],
             'gbm__min_samples_leaf': [10, 15]}

In [ ]:
#  setup gridsearchcv with estimator, param_grid
GBRGrid = GridSearchCV(estimator = GBRPipe,param_grid=param_grid)

In [ ]:
GBRGrid.fit(X_train,y_train)

In [ ]:
gc.collect()

In [ ]:
print('Validation R^2: %.5f'  % r2_score(y_val,GBRGrid.predict(X_val)))
print('Validation RMSE: %.5f\n'   % mean_squared_error(y_val,GBRGrid.predict(X_val)))
print('Validation Mean Absolute Error: %.5f\n'   % mean_absolute_error(y_val,GBRGrid.predict(X_val)))

In [ ]:
GBRGrid.predict(test)

In [ ]:
GBRGrid.best_score_

In [ ]:
predicted = GBRGrid.predict(test)
predicted

In [ ]:
test_result = test
test_result['pressure'] = predicted

In [ ]:
submit_result = test_result[['id','pressure']]
submit_result

In [ ]:
submit_result.to_csv('/content/drive/MyDrive/Colab Notebooks/Predictive_Data/vantilator_google/submission.csv', index=False)